In [14]:
import numpy as np

In [15]:
with np.load("./mnist.npz", allow_pickle=True) as f:
    X_train, y_train = f["x_train"], f["y_train"]
    X_test, y_test = f["x_test"], f["y_test"]

print('X_train: ' + str(X_train.shape))
print('Y_train: ' + str(y_train.shape))
print('X_test:  ' + str(X_test.shape))
print('Y_test:  ' + str(y_test.shape))

X_train: (60000, 28, 28)
Y_train: (60000,)
X_test:  (10000, 28, 28)
Y_test:  (10000,)


In [16]:
X_train.shape

(60000, 28, 28)

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [18]:
import torch.nn.functional as F

In [19]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)  # Assuming 10 classes for classification

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [20]:
# Assuming `data` is your numpy dataset of shape (60000, 28, 28)
X_train = X_train.reshape((60000, 1, 28, 28))  # Add channel dimension

dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [21]:
model = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [22]:
num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in dataloader:
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}')


Epoch 1, Loss: 0.2551312198947325
Epoch 2, Loss: 0.055607871672295944
Epoch 3, Loss: 0.04143442150884505
Epoch 4, Loss: 0.035300032492047044
Epoch 5, Loss: 0.028310701437648973
Epoch 6, Loss: 0.026066894085841397
Epoch 7, Loss: 0.02224493855974404
Epoch 8, Loss: 0.019357073930584303
Epoch 9, Loss: 0.02025179291832149
Epoch 10, Loss: 0.01850188865909174
